In [41]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import random 

In [20]:
# Carrega o data set
data = load_iris()

# Atribui valores à entrada e à saída 
X = data.data
y = data.target

In [26]:
# Separando dados para treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Dados para treinamento: {len(X_train)}\nDados para teste: {len(X_test)}')

Dados para treinamento: 120
Dados para teste: 30


### PARÂMETROS GLOBAIS

In [33]:
CLASS_NUM = 3
POPULATION_SIZE = 60

X_train

array([[4.6, 3.6, 1. , 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [6.7, 3.1, 4.4, 1.4],
       [4.8, 3.4, 1.6, 0.2],
       [4.4, 3.2, 1.3, 0.2],
       [6.3, 2.5, 5. , 1.9],
       [6.4, 3.2, 4.5, 1.5],
       [5.2, 3.5, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.2, 4.1, 1.5, 0.1],
       [5.8, 2.7, 5.1, 1.9],
       [6. , 3.4, 4.5, 1.6],
       [6.7, 3.1, 4.7, 1.5],
       [5.4, 3.9, 1.3, 0.4],
       [5.4, 3.7, 1.5, 0.2],
       [5.5, 2.4, 3.7, 1. ],
       [6.3, 2.8, 5.1, 1.5],
       [6.4, 3.1, 5.5, 1.8],
       [6.6, 3. , 4.4, 1.4],
       [7.2, 3.6, 6.1, 2.5],
       [5.7, 2.9, 4.2, 1.3],
       [7.6, 3. , 6.6, 2.1],
       [5.6, 3. , 4.5, 1.5],
       [5.1, 3.5, 1.4, 0.2],
       [7.7, 2.8, 6.7, 2. ],
       [5.8, 2.7, 4.1, 1. ],
       [5.2, 3.4, 1.4, 0.2],
       [5. , 3.5, 1.3, 0.3],
       [5.1, 3.8, 1.9, 0.4],
       [5. , 2. , 3.5, 1. ],
       [6.3, 2.7, 4.9, 1.8],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [5.6, 2

### GERAÇÃO DA POPULAÇÃO DE ANTICORPOS

In [28]:
# Classe Anticorpo

class Antibody:
    def __init__(self, vector, label):
        self.vector = vector
        self.label = label
        self.affinity = None

Para a geração da população, iremos escolher aleatoriamente os valores do vetor, dentro de um limite estimado pelo menor e maior valor de cada coluna do dataset.

Por fim, iremos atribuir igualmente as labels para cada anticorpo,de modo que cada classe tenha a mesma quantidade de anticorpos gerados.

In [40]:
# Extraindo o valor minimo e máximo de cada atributo da Iris

min_sepal_length = min(X_train[:, 0])
max_sepal_length = max(X_train[:, 0])

min_sepal_width = min(X_train[:, 1])
max_sepal_width = max(X_train[:, 1])

min_petal_length = min(X_train[:, 2])
max_petal_length = max(X_train[:, 2])

min_petal_width = min(X_train[:, 3])
max_petal_width = max(X_train[:, 3])

print(f'Limite do comprimento da sépala: [{min_sepal_length}, {max_sepal_length}]\nLimite da largura da sépala: [{min_sepal_width}, {max_sepal_width}]\nLimite do comprimento da pétala: [{min_petal_length}, {max_petal_length}]\nLimite da largura da sépala: [{min_petal_width}, {max_petal_width}]\n')

Limite do comprimento da sépala: [4.3, 7.7]
Limite da largura da sépala: [2.0, 4.4]
Limite do comprimento da pétala: [1.0, 6.7]
Limite da largura da sépala: [0.1, 2.5]



In [ ]:
# Método para gerar um anticorpo

def generateAntibody(label):

    # Escolhe um valor aleatorio dentro da faixa estimada
    vector = [
        random.uniform(min_sepal_length, max_sepal_length),
        random.uniform(min_sepal_width, max_sepal_width),
        random.uniform(min_petal_length, max_sepal_length),
        random.uniform(min_petal_width, max_petal_width)
    ]
    
    antibody = Antibody(vector, label)
    return antibody

In [47]:
# Método para criar a população com base no seu tamanho e numero de classes

def generatePopulation():

    population = []
    # Cria labels igualmente com base no numero de classes
    labels = [i % CLASS_NUM for i in range(POPULATION_SIZE)]

    for i in range(POPULATION_SIZE):
        population.append(generateAntibody(labels[i]))
    
    return population

In [52]:
pop = generatePopulation()

for i in pop:
    print(i.label)

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
